# Creating flows in jobflow

In this tutorial, you will:

- Learn about the `Flow` object.
- Set the configuration settings of a flow and its jobs.

The purpose of this tutorial is to delve into the basic functionality of flows and gain a feeling for what is possible. Later tutorials will describe how to create dynamic flows.

## Creating job objects

The building block of jobflows are `Job` objects. Jobs are delayed calls to python functions whose outputs are stored in a database. The easiest way to create a job is using the `@job` decorator. The job decorator can be applied to any function, even those with optional parameters.


In [39]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

We will start by defining two simple jobs that we will stitch together into a flow.


In [40]:
from jobflow import job

@job
def add(a, b, c=2):
    return a + b + c

@job
def mult(a, b):
    return a * b

To combine these two `Job` objects into a single workflow, we can take advantage of the `Flow` constructor in Jobflow.


In [41]:
from jobflow import Flow

job1 = add(1, 2)
job2 = mult(job1.output, 3)

flow = Flow([job1, job2], name="my-flow")

Because `job2` depends on the output of `job1`, it will only run if/when `job1` completes successfully. Jobflow will automatically determine the connectivity of the jobs and run them accordingly. Here, we have also given the flow an optional name, which can be useful for tracking purposes.


## Setting Metadata


It can often be useful to attach metadata to jobs and flows before they are run, particularly for querying purposes later on. This can be done with the `update_metadata` function. The `name` can also be updated on-the-fly.


In [42]:
job1 = add(1, 2)

job1.name = "test"
job1.update_metadata({"tags": ["test"]})

## Running Flows

We can run the flow locally by calling the `run_locally` function.


In [43]:
from jobflow.managers.local import run_locally

responses = run_locally(flow)

2023-06-07 23:15:11,193 INFO Started executing jobs locally
2023-06-07 23:15:11,213 INFO Starting job - add (8ed01a13-ef87-4892-9522-dde3359c7ea8)
2023-06-07 23:15:11,226 INFO Finished job - add (8ed01a13-ef87-4892-9522-dde3359c7ea8)
2023-06-07 23:15:11,227 INFO Starting job - mult (227f623f-4059-4b35-84e0-50fcbb67d08e)
2023-06-07 23:15:11,236 INFO Finished job - mult (227f623f-4059-4b35-84e0-50fcbb67d08e)
2023-06-07 23:15:11,237 INFO Finished executing jobs locally


The output contains a UUID for each `job` in the flow along with the outputs for each job.


In [44]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

8ed01a13-ef87-4892-9522-dde3359c7ea8 -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
227f623f-4059-4b35-84e0-50fcbb67d08e -> {1: Response(output=15, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


The UUID can also be obtained from the `job` object directly, which is useful for indexing the output.


In [45]:
print(responses[job2.uuid][1].output)

15
